# Language Response Asymmetry (Cohere)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import cohere

load_dotenv()

COHERE_API_KEY = os.environ.get("COHERE_API_KEY")
if not COHERE_API_KEY:
    raise RuntimeError("Missing COHERE_API_KEY in environment/.env")

co = cohere.ClientV2(api_key=COHERE_API_KEY)

In [3]:
# Generation models (examples; adjust to what your grant enables)
MODEL_AYA = "c4ai-aya-expanse-8b"  # main model
MODEL_COMMAND = "command-r"   # optional baseline

# Embedding model
MODEL_EMBED = "embed-multilingual-v3.0"

In [4]:
CONDITIONS = [
    {"name": "aya_en", "model": MODEL_AYA, "lang": "en"},
    {"name": "aya_fr", "model": MODEL_AYA, "lang": "fr"},
    # later:
    # {"name": "aya_it", "model": MODEL_AYA, "lang": "it"},
    # {"name": "aya_sw", "model": MODEL_AYA, "lang": "sw"},
]

CONDITION_PAIRS = [
    ("aya_en", "aya_fr"),
    # later:
    # ("aya_en", "aya_it"),
    # ("aya_en", "aya_sw"),
]

In [5]:
# Number of stochastic samples per (prompt, condition)
N_SAMPLES_PER_CONDITION = int(os.environ.get("N_SAMPLES_PER_CONDITION", 2))

# Number of random projection directions for sliced KS
N_DIRECTIONS = int(os.environ.get("N_DIRECTIONS", 64))

# Random seed for reproducibility
RANDOM_STATE = int(os.environ.get("RANDOM_STATE", 12345))

In [6]:
from typing import Dict, List

def cohere_generate_answers(
    prompt_en: str,
    prompt_fr: str,
    condition: Dict,
    n_samples: int = 8,
    max_tokens: int = 128,
    temperature: float = 0.7,
    top_p: float = 0.9,
) -> List[str]:
    lang = condition["lang"]
    model = condition["model"]
    prompt = prompt_en if lang == "en" else prompt_fr

    outputs = []
    for _ in range(n_samples):
        resp = co.chat(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            p=top_p,                 # Cohere uses `p` for top-p in many examples/docs
            max_tokens=max_tokens,
        )
        # v2 responses can vary by SDK version; this is the common pattern:
        text = resp.message.content[0].text if hasattr(resp, "message") else resp.text
        outputs.append(text)

    return outputs

In [7]:
import numpy as np

def cohere_embed_texts(
    texts: List[str],
    model: str = MODEL_EMBED,
    input_type: str = "search_document",
) -> np.ndarray:
    inputs = [{"content": [{"type": "text", "text": t}]} for t in texts]

    resp = co.embed(
        model=model,
        inputs=inputs,
        input_type=input_type,
        embedding_types=["float"],
    )

    # Common: resp.embeddings.float is a list[list[float]]
    emb = resp.embeddings.float
    return np.asarray(emb, dtype=np.float32)

In [15]:
test_prompt_en = "Who discovered penicillin?"
test_prompt_fr = "Qui a découvert la pénicilline ?"

answers = cohere_generate_answers(test_prompt_en, test_prompt_fr, CONDITIONS[0], n_samples=2)
print("Sample answers:", answers)

E = cohere_embed_texts(answers)
print("Embeddings shape:", E.shape)

Sample answers: ['Penicillin was discovered by Alexander Fleming, a Scottish biologist, and pharmacist, in 1928. Fleming noticed that mold, specifically Penicillium notatum, had grown in a culture dish of Staphylococcus bacteria, causing the bacteria to deteriorate. This observation led him to conclude that the mold produced a substance that could kill or inhibit the growth of bacteria.\n\nFleming\'s discovery was a significant milestone in the field of medicine and marked the beginning of the antibiotic era. He named the substance "penicillin" and shared his findings with other scientists, including Howard Florey and Ernst Chain, who later played crucial roles', "Penicillin was discovered by Alexander Fleming, a Scottish bacteriologist, in 1928. Fleming noticed that a mold, later identified as Penicillium notatum, had grown in a petri dish containing Staphylococcus bacteria, causing the bacteria to deteriorate and die. This observation led to the understanding that the mold produced a

In [8]:
from stats_helpers import sliced_ks_distance, symmetry_from_sks

generate_answers_for_condition = cohere_generate_answers
embed_texts = cohere_embed_texts

In [9]:

from tqdm.auto import tqdm
import pandas as pd

from prompts import PROMPTS


def build_condition_lookup(conditions: List[Dict]) -> Dict[str, Dict]:
    return {c["name"]: c for c in conditions}



condition_lookup = build_condition_lookup(CONDITIONS)

results = []

for prompt in tqdm(PROMPTS, desc="Prompts"):
    prompt_id = prompt["id"]
    prompt_type = prompt.get("type", "unknown")
    en_text = prompt["en"]
    fr_text = prompt["fr"]

    # Cache answers and embeddings per condition
    answers_by_condition: Dict[str, List[str]] = {}
    emb_by_condition: Dict[str, np.ndarray] = {}

    # Generate answers and embeddings for each condition
    for cond in CONDITIONS:
        cond_name = cond["name"]
        answers = generate_answers_for_condition(
            prompt_en=en_text,
            prompt_fr=fr_text,
            condition=cond,
            n_samples=N_SAMPLES_PER_CONDITION,
        )
        answers_by_condition[cond_name] = answers
        emb_by_condition[cond_name] = embed_texts(answers)

    # Now compute symmetry metrics for each condition pair
    for cond_a, cond_b in CONDITION_PAIRS:
        emb_A = emb_by_condition[cond_a]
        emb_B = emb_by_condition[cond_b]

        sks_metrics = sliced_ks_distance(
            emb_A,
            emb_B,
            n_directions=N_DIRECTIONS,
            random_state=RANDOM_STATE,
        )
        sym = symmetry_from_sks(sks_metrics)

        results.append(
            {
                "prompt_id": prompt_id,
                "prompt_type": prompt_type,
                "condition_A": cond_a,
                "condition_B": cond_b,
                "ks_mean": sks_metrics.mean,
                "ks_std": sks_metrics.std,
                "ks_sem": sks_metrics.sem,
                "ks_ci_low": sks_metrics.ci_low,
                "ks_ci_high": sks_metrics.ci_high,
                "sym_mean": sym["sym_mean"],
                "sym_std": sym["sym_std"],
                "sym_sem": sym["sym_sem"],
                "sym_ci_low": sym["sym_ci_low"],
                "sym_ci_high": sym["sym_ci_high"],
            }
        )

results_df = pd.DataFrame(results)
results_df


/Users/disipio/.local/share/virtualenvs/multilingual-llm-symmetry-UDP034c6/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Prompts:  83%|████████▎ | 5/6 [00:39<00:07,  7.83s/it]


TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': '43ac95f5f2b3db48e12c881c6062cef9', 'date': 'Mon, 22 Dec 2025 21:28:16 GMT', 'x-envoy-upstream-service-time': '5', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 429, body: {'id': '093b6f7b-bafe-45ef-a6f9-fe58251ff4ef', 'message': "You are using a Trial key, which is limited to 20 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@